In [50]:
import pandas as pd
import os as os
import mendeleev as ptable
import pymatgen as pmtg
import pymatgen.io.vasp.sets
import pymatgen.core
import numpy as np
import sympy as sympy
import scipy as scipy
import random as random
import itertools as itertools
import csv as csv
import xyz_py as xyz
import tarfile as tarfile
from utils import *
from Determine_Bonds_Get_Input import find_orbitals_get_characteristics, build_graph_tensor

In [5]:
class g16_input_file:
    def __init__(self, seed_structure_directory, molecular_formula, job_type, input_directory, output_directory, AO_basis, polarization, job_title, SCF_optimization, csv_filepath, record_filepath, record_filename, element, n_CPU, allowed_memory):
        self.seed_structure_directory = seed_structure_directory
        self.molecular_formula = molecular_formula
        self.job_type = job_type
        self.input_directory = input_directory
        self.output_directory = output_directory
        self.AO_basis = AO_basis
        self.polarization = polarization
        self.job_title = job_title
        self.SCF_optimization = SCF_optimization
        self.csv_filepath = csv_filepath
        self.record_filepath = record_filepath
        self.record_filename = record_filename
        self.element = element
        self.n_CPU = n_CPU
        self.allowed_memory = allowed_memory

    def generate_input_file(self, molecular_formula, job_type, input_directory, output_directory, seed_structure_directory, csv_filepath, record_filepath, record_filename, n_CPU, allowed_memory):
        input_filepath = os.path.join(input_directory, molecular_formula)
        input_filename = molecular_formula
        input_file = open(input_filepath, "w")
        seed_structure = seed_structure_directory + "/" + molecular_formula + ".csv"
        energy_checkpoint_path = os.path.join(output_directory, molecular_formula)
        geom_checkpoint_path = os.path.join(output_directory, molecular_formula + "_geom")
        energy_chk = open(energy_checkpoint_path, "w")
        geom_chk = open(geom_checkpoint_path, "w")
        energy_chk_name = str(molecular_formula) + ".chk"
        geom_chk_name = str(molecular_formula) + "_geom" + ".chk"
        if job_type == "ground_state_energy_calculation":
            job_abbrev = "SP"
            input_file.write("%chk=" + energy_chk_name)
            input_file.write("\n")
        elif job_type == "geometry_optimization":
            job_abbrev = "Opt"
            input_file.write("%chk=" + geom_chk_name)
            input_file.write("\n")
        graph_tensor = build_graph_tensor(csv_filepath, record_filepath, record_filename)
        molecular_characteristics = find_orbitals_get_characteristics(molecular_formula = molecular_formula, seed_structure_directory = seed_structure_directory, element = None, graph_tensor = graph_tensor)
        symbol, Z, block = molecular_characteristics.get_symbols_and_numbers(molecular_formula = molecular_characteristics.molecular_formula, seed_structure_directory = molecular_characteristics.seed_structure_directory, graph_tensor = molecular_characteristics.graph_tensor)
        if "d" in block or "f" in block:
            AO_basis = "triple_zeta"
        elif "C" in symbol:
            AO_basis = "6-31G"
        elif "Na" in symbol or "K" in symbol or "Mg" in symbol or "Ca" in symbol or "Li" in symbol or "Be" in symbol:
            AO_basis = "LANL_double_zeta"
        else:
            AO_basis = "STO-3G"
        input_file.write("%nprocs=" + str(n_CPU))
        input_file.write("\n")
        input_file.write("%mem=" + allowed_memory)
        input_file.write("\n")
        if AO_basis == "triple_zeta":
            if polarization == True:
                input_file.write("# 3-21G**" + " " + job_abbrev + " " + SCF_optimization)
                input_file.write("\n")
            else:
                input_file.write("# 3-21G" + " " + job_abbrev + " " + SCF_optimization)
                input_file.write("\n")
        elif AO_basis == "6-31G":
            if polarization == True:
                input_file.write("# 6-31G**" + " " + job_abbrev + " " + SCF_optimization)
                input_file.write("\n")
            else: 
                input_file.write("# " + AO_basis + " " + job_abbrev + " " + SCF_optimization)
                input_file.write("\n")
        elif AO_basis == "LANL_double_zeta":
            input_file.write("LANL2DZ" + " " + job_abbrev + " " + SCF_optimization)
            input_file.write("\n")
        input_file.write(job_title)
        input_file.write("\n")
        input_file.write(str(0) + "  " + str(1))
        atomic_pos = pd.read_csv(seed_structure)
        labels = atomic_pos.insert(0, "labels", symbol, True)
        atomic_pos = atomic_pos.to_numpy()
        atomic_pos = atomic_pos[1:]
        labels = labels[1:].to_numpy()
        for i in np.arange(0, len(atomic_pos.transpose())):
            coordinate_specification = " " + str(labels.transpose()[0][i])
            for j in np.arange(0, len(atomic_pos.transpose()[i])):
                coordinate_specification = coordinate_specification + " " + str(atomic_pos[i][j])
            input_file.write(coordinate_specification)
            input_file.write("\n")
        return input_filename
        
        
        

In [71]:
class VASP_input:
    def __init__(self, working_directory, molecular_formula, job_type, seed_structure_directory, csv_filepath, record_filepath, record_filename, pseudopotential, xc_functional, cluster_type, principal_lattice_vector, accuracy_bound, max_iterations, complex_rotation_term, rotation_angle, n_atoms, average_bond_length, phase, input_format, run_calculation):
        self.working_directory = working_directory
        self.molecular_formula = molecular_formula
        self.job_type = job_type
        self.seed_structure_directory = seed_structure_directory
        self.csv_filepath = csv_filepath
        self.record_filepath = record_filepath
        self.record_filename = record_filename
        self.pseudopotential = pseudopotential
        self.xc_functional = xc_functional
        self.cluster_type = cluster_type
        self.principal_lattice_vector = principal_lattice_vector
        self.accuracy_bound = accuracy_bound
        self.max_iterations = max_iterations
        self.complex_rotation_term = complex_rotation_term
        self.rotation_angle = rotation_angle
        self.n_atoms = n_atoms
        self.average_bond_length = average_bond_length
        self.phase = phase
        self.input_format = input_format
        self.run_calculation = run_calculation

    def convert_rotation_to_matrix(self, complex_rotation_term, rotation_angle):
        rotation_str_representation = str(complex_rotation_term)
        if rotation_str_representation[0] == '[':
            rotation_obj = scipy.spatial.transform.Rotation.from_quat(complex_rotation_term)
            rotation_matrix = rotation_obj.as_matrix()
        else:
            rotation_matrix = np.array([[sympy.cos(rotation_angle), -sympy.sin(rotation_angle)], [sympy.sin(rotation_angle), sympy.cos(rotation_angle)]])
        return rotation_matrix
    
    def find_mapping_to_coordinate_axis(self, principal_lattice_vector, accuracy_bound, max_iterations): 
        orthogonal_shift = principal_lattice_vector[2]
        rotation_steps = []
        starting_vector = principal_lattice_vector
        update_index = 0
        n_transformations = 0
        while n_transformations < max_iterations:
            lattice_norm = np.sqrt(principal_lattice_vector[0]**2 + principal_lattice_vector[1]**2 + principal_lattice_vector[1]**2)
            xy_trace = np.array([principal_lattice_vector[0], principal_lattice_vector[1], 0])
            trace_norm = np.sqrt(xy_trace[0]**2 + xy_trace[1]**2 + xy_trace[2]**2)
            xy_trace = np.array([xy_trace[0] * (lattice_norm / trace_norm), xy_trace[1] * (lattice_norm / trace_norm), xy_trace[2] * (lattice_norm / trace_norm)])
            a = sympy.symbols('a', real = True)
            solution_space = sympy.solve(lattice_norm**2 * sympy.cos(a) - np.dot(principal_lattice_vector, xy_trace), a)
            if len(solution_space) > 0:
                angular_shift = (2 * np.pi) - solution_space[0] 
                axis_of_rotation = np.cross(principal_lattice_vector, xy_trace)
                rotation_quat = np.array([sympy.cos(angular_shift / 2), axis_of_rotation[0] * sympy.sin(angular_shift / 2), axis_of_rotation[1] * sympy.sin(angular_shift / 2), axis_of_rotation[2] * sympy.sin(angular_shift / 2)])
                rotation_matrix = self.convert_rotation_to_matrix(complex_rotation_term = rotation_quat, rotation_angle = angular_shift)
                checkpoint = principal_lattice_vector
                checkpoint_matrix = rotation_matrix
                principal_lattice_vector = principal_lattice_vector @ rotation_matrix
                new_shift = principal_lattice_vector[2]
                if abs(new_shift) < abs(orthogonal_shift):
                    rotation_steps.append(rotation_matrix)
                    n_transformations += 1
                    orthogonal_shift = new_shift
                else:
                    angular_shift = -1 * ((2 * np.pi) - solution_space[0])
                    axis_of_rotation = np.cross(checkpoint, xy_trace)
                    rotation_quat = np.array([sympy.cos(angular_shift / 2), axis_of_rotation[0] * sympy.sin(angular_shift / 2), axis_of_rotation[1] * sympy.sin(angular_shift / 2), axis_of_rotation[2] * sympy.sin(angular_shift / 2)])
                    rotation_matrix = self.convert_rotation_to_matrix(complex_rotation_term = rotation_quat, rotation_angle = angular_shift)
                    principal_lattice_vector = checkpoint @ rotation_matrix
                    new_shift = principal_lattice_vector[2]
                    n_transformations += 1
                    if abs(new_shift) < abs(orthogonal_shift):
                        rotation_steps.append(rotation_matrix)
                        n_transformations += 1
                        orthogonal_shift = new_shift
                    else:
                        if orthogonal_shift > 0:
                            variation_angle = -1 * (np.pi / 180)
                        else:
                            variation_angle = np.pi / 180
                        axis_of_rotation = np.cross(checkpoint, xy_trace)
                        checkpoint_norm = sympy.sqrt(checkpoint[0]**2 + checkpoint[1]**2 + checkpoint[2]**2)
                        variation_quat = np.array([sympy.cos(variation_angle / 2), axis_of_rotation[0] * sympy.sin(variation_angle / 2), axis_of_rotation[1] * sympy.sin(variation_angle / 2), axis_of_rotation[2] * sympy.sin(variation_angle / 2)])
                        variation_matrix = self.convert_rotation_to_matrix(complex_rotation_term = variation_quat, rotation_angle = variation_angle)
                        varied_components = checkpoint @ variation_matrix
                        rotation_steps.append(variation_matrix)
                        principal_lattice_vector = varied_components
                        orthogonal_shift = principal_lattice_vector[2]
                        n_transformations += 1
            if abs(orthogonal_shift) >= abs(accuracy_bound):
               continue
            else:
                break  
            
        flattening_rotation = np.eye(len(rotation_steps[0]))
        for i in np.arange(0, len(rotation_steps)):
            flattening_rotation = flattening_rotation @ rotation_steps[i]
        u, s, v = scipy.linalg.svd(flattening_rotation)
        reciporical_singular_vals = np.eye(len(s))
        for i in np.arange(0, len(s)):
            for j in np.arange(0, len(s)):
                if reciporical_singular_vals[i][j] == 1:
                    reciporical_singular_vals[i][j] = 1 / s[j]
        u_inv = np.linalg.inv(u)
        v_inv_t = np.linalg.inv(v).transpose()
        reverse_flattening_rotation = u_inv @ reciporical_singular_vals @ v_inv_t
        prev_lattice_vector = principal_lattice_vector
        principal_lattice_vector = np.array([prev_lattice_vector[0], prev_lattice_vector[1]])
        planar_shift = principal_lattice_vector[1]
        rotation_steps = []
        update_index = 0
        planar_transformations = 0
        while planar_transformations < max_iterations:
            planar_norm = np.sqrt(principal_lattice_vector[0]**2 + principal_lattice_vector[1]**2)
            unit_cell_axis = np.array([principal_lattice_vector[0], 0])
            cell_axis_norm = np.sqrt(unit_cell_axis[0]**2 + unit_cell_axis[1]**2)
            unit_cell_axis = np.array([unit_cell_axis[0] * (planar_norm / cell_axis_norm), unit_cell_axis[1] * (planar_norm / cell_axis_norm)])
            a = sympy.symbols('a', real = True)
            solution_space = sympy.solve(planar_norm**2 * sympy.cos(a) - np.dot(principal_lattice_vector, unit_cell_axis), a)
            if len(solution_space) > 0:
                xy_angular_shift = (2 * np.pi) - solution_space[0] 
                axis_of_rotation = np.array([0,0,1])
                imag_term = sympy.I
                rotation_number = sympy.cos(xy_angular_shift) + sympy.sin(xy_angular_shift) * imag_term
                rotation_matrix = self.convert_rotation_to_matrix(complex_rotation_term = rotation_number, rotation_angle = xy_angular_shift)
                checkpoint = principal_lattice_vector
                checkpoint_matrix = rotation_matrix
                principal_lattice_vector = principal_lattice_vector @ rotation_matrix
                new_shift = principal_lattice_vector[1]
                if abs(new_shift) < abs(planar_shift):
                    rotation_steps.append(rotation_matrix)
                    planar_transformations += 1
                    planar_shift = new_shift
                else:
                    xy_angular_shift = -1 * ((2 * np.pi) - solution_space[0])
                    axis_of_rotation = np.array([0,0,1])
                    rotation_number = sympy.cos(xy_angular_shift) + sympy.sin(xy_angular_shift) * imag_term
                    rotation_matrix = self.convert_rotation_to_matrix(complex_rotation_term = rotation_number, rotation_angle = xy_angular_shift)
                    principal_lattice_vector = checkpoint @ rotation_matrix
                    new_shift = principal_lattice_vector[1]
                    n_transformations += 1
                    if abs(new_shift) < abs(planar_shift):
                        rotation_steps.append(rotation_matrix)
                        planar_transformations += 1
                        planar_shift = new_shift
                    else:
                        if planar_shift > 0:
                            variation_angle = -1 * (np.pi / 180)
                        else:
                            variation_angle = np.pi / 180
                        axis_of_rotation = np.array([0,0,1])
                        checkpoint_norm = sympy.sqrt(checkpoint[0]**2 + checkpoint[1]**2)
                        variation_number = np.cos(variation_angle) + np.sin(variation_angle) * imag_term
                        variation_matrix = self.convert_rotation_to_matrix(complex_rotation_term = variation_number, rotation_angle = variation_angle)
                        varied_components = checkpoint @ variation_matrix
                        rotation_steps.append(variation_matrix)
                        principal_lattice_vector = varied_components
                        planar_shift = principal_lattice_vector[1]
                        planar_transformations += 1
            if abs(planar_shift) >= abs(accuracy_bound):
                continue
            else:
                break  

        unit_cell_axis = principal_lattice_vector
        axis_alignment = sympy.eye(len(rotation_steps[0]))
        for i in np.arange(0, len(rotation_steps)):
            axis_alignment = axis_alignment @ sympy.Matrix(rotation_steps[i])
        u, s, v = axis_alignment.singular_value_decomposition()
        s = np.diag(np.array(s))
        reciporical_singular_vals = np.eye(len(s))
        for i in np.arange(0, len(s)):
            for j in np.arange(0, len(s)):
               if reciporical_singular_vals[i][j] == 1:
                    reciporical_singular_vals[i][j] = 1 / s[j]
        reciporical_singular_vals = sympy.Matrix(reciporical_singular_vals)
        u_inv = u.inv()
        v_inv_t = u.inv().transpose()
        reverse_axis_alignment = u_inv @ reciporical_singular_vals @ v_inv_t
        initial_principal_norm = sympy.sqrt(starting_vector[0]**2 + starting_vector[1]**2 + starting_vector[2]**2)
        aligned_principal_norm = sympy.sqrt(principal_lattice_vector[0]**2 + principal_lattice_vector[1]**2 + prev_lattice_vector[2]**2)
        principal_lattice_vector = np.array([unit_cell_axis[0], unit_cell_axis[1], prev_lattice_vector[2]])
        principal_lattice_vector = np.array([principal_lattice_vector[0] * (initial_principal_norm / aligned_principal_norm), principal_lattice_vector[1] * (initial_principal_norm / aligned_principal_norm), principal_lattice_vector[2] * (initial_principal_norm / aligned_principal_norm)])
        return flattening_rotation, reverse_flattening_rotation, axis_alignment, reverse_axis_alignment, initial_principal_norm, aligned_principal_norm

   # def 

   
    def generate_cluster_object(self, seed_structure_directory, molecular_formula, cluster_type, csv_filepath, record_filepath, record_filename, accuracy_bound, max_iterations, n_atoms, average_bond_length, xc_functional, phase, input_format, run_calculation):
        seed_structure = seed_structure_directory + "/" + molecular_formula + ".csv"
        if cluster_type == "molecule":
            graph_tensor = build_graph_tensor(csv_filepath, record_filepath, record_filename)
            molecular_characteristics = find_orbitals_get_characteristics(molecular_formula = molecular_formula, seed_structure_directory = seed_structure_directory, element = None, graph_tensor = graph_tensor)
            symbol, Z, block = molecular_characteristics.get_symbols_and_numbers(molecular_formula = molecular_characteristics.molecular_formula, seed_structure_directory = molecular_characteristics.seed_structure_directory, graph_tensor = molecular_characteristics.graph_tensor)
            coordinate_vectors = []
            species_in_molecule = []
            for i in np.arange(0, len(symbol)):
                species_in_molecule.append(str(symbol[i]))
            atomic_pos_components = pd.read_csv(seed_structure)
            atomic_pos_components = atomic_pos_components[:len(graph_tensor[1])]
            atomic_pos_components = atomic_pos_components.to_numpy()
            atomic_pos_components = atomic_pos_components.transpose()[0:4]
            atomic_pos_components = atomic_pos_components.transpose()
            for j in np.arange(0, len(atomic_pos_components)):
                coordinate_vectors.append(atomic_pos_components[j][1:])
            molecule = pmtg.core.structure.Molecule(species_in_molecule, coordinate_vectors)
            cluster = molecule
        elif cluster_type == "periodic":
            if phase == "solid":
                vacuum_between_cells = False
                bravais_lattice = pd.read_csv(seed_structure)
                bravais_lattice = bravais_lattice.to_numpy()
                bravais_lattice = bravais_lattice[1:n_atoms]
                bravais_lattice = bravais_lattice.transpose()[1:4]
                bravais_lattice = bravais_lattice.transpose()
                translation_factor = bravais_lattice[0]
                unit_cell_size_estimate = 1.633 * average_bond_length
                for i in np.arange(0, len(bravais_lattice)):
                    bravais_lattice[i] = bravais_lattice[i] + translation_factor
                representative_unit_cell = 0
                species_index = 0
                cell_norms = np.array([])
                representative_norms = np.array([])
                for j in np.arange(0, len(bravais_lattice)):
                    representative_squares = np.array([])
                    for k in np.arange(0, len(bravais_lattice[representative_unit_cell + species_index])):
                        representative_squares = np.append(representative_squares, (bravais_lattice[representative_unit_cell + species_index][k] - bravais_lattice[j][k])**2)
                    representative_norms = np.append(representative_norms, np.sqrt(np.sum(representative_squares)))
                i = 1
                nearest_neighbor_indices = np.array([])
                refernece_unit_cell_norm = representative_norms[random.randint(1,3)]
                for i in np.arange(1, len(representative_norms)):
                    if representative_norms[i] <= unit_cell_size_estimate and representative_norms[i] < 1.5 * representative_norms[0]:
                        nearest_neighbor_indices = np.append(nearest_neighbor_indices, i)
                    else:
                        break
                n_nearest_neighbors = len(nearest_neighbor_indices)
                nearest_neighbors = bravais_lattice[nearest_neighbor_indices[0]:nearest_neighbor_indices[n_nearest_neighbors]]
                translational_symmetry_flags = np.zeros(len(nearest_neighbor_indices))
                while True:
                    guess_symmetry_dir = nearest_neighbors[random.randint(1, n_nearest_neighbors)]
                    for j in np.arange(0, len(nearest_neighbors)):
                        adjusted_j_point = nearest_neighbors[j] + guess_symmetry_dir
                        for k in np.arange(0, len(bravais_lattice)):
                            if adjusted_j_point[0] == bravais_lattice[k][0] and adjusted_j_point[1] == bravais_lattice[k][1] and adjusted_j_point[0] == bravais_lattice[k][2]:
                                translational_symmetry_flags[k] == 1
                    if 0 not in translational_symmetry_flags:
                        principal_lattice_vector = guess_symmetry_dir
                        break
                    else:
                        guess_symmetry_dir = nearest_neighbors[random.randint(1, n_nearest_neighbors)]
                        continue                
                xy_plane_rotation, xy_plane_reverse, coordinate_axis_alignment, coordinate_axis_reverse, lattice_vector_norm, aligned_vector_norm = self.find_mapping_to_coordinate_axis(principal_lattice_vector, accuracy_bound, max_iterations)
                principal_vector_planar = principal_lattice_vector @ xy_plane_rotation
                principal_lattice_vector = np.array([principal_vector_planar[0], principal_vector_planar[1]])
                principal_vector_aligned = principal_lattice_vector @ coordinate_axis_alignment
                principal_lattice_vector = np.array([principal_vector_aligned[0], principal_vector_aligned[1], principal_vector_planar[2]])
                principal_lattice_vector = np.array([principal_lattice_vector[0] * (lattice_vector_norm / aligned_vector_norm), principal_lattice_vector[1] * (lattice_vector_norm / aligned_vector_norm), principal_lattice_vector[2] * (lattice_vector_norm / aligned_vector_norm)])
                aligned_unit_cell = []
                for k in range(0, len(cell_norms)):
                    lattice_vector = np.array([cell_coordinates[k][0], cell_coordinates[k][1], cell_coordinates[k][2]])
                    norm = np.sqrt(lattice_vector[0]**2 + lattice_vector[1]**2 + lattice_vector[2]**2)
                    planar = lattice_vector @ xy_plane_rotation
                    lattice_vector = np.array([planar[0], planar[1]])
                    aligned = lattice_vector @ coordinate_axis_alignment
                    lattice_vector = np.array([aligned[0], aligned[1], planar[2]])
                    aligned_norm = np.sqrt(lattice_vector[0]**2 + lattice_vector[1]**2 + lattice_vector[2]**2)
                    lattice_vector = np.array([lattice_vector[0] * (norm / aligned_norm), lattice_vector[1] * (norm / aligned_norm), lattice_vector[2] * (norm / aligned_norm)])
                    aligned_unit_cell.append(lattice_vector)
                aligned_cell_angles = np.array([])
                principal_norm = np.sqrt(principal_lattice_vector[i][0]**2 + principal_lattice_vector[i][1]**2 + principal_lattice_vector[i][2]**2)
                for i in range(0, len(aligned_unit_cell)):
                    i_norm = np.sqrt(aligned_unit_cell[i][0]**2 + aligned_unit_cell[i][1]**2 + aligned_unit_cell[i][2]**2) 
                    i_angle = np.dot(aligned_unit_cell[i], principal_lattice_vector) / (i_norm * principal_norm)
                    if i_angle < 0:
                        if np.dot(aligned_unit_cell[i], principal_lattice_vector) < 0:
                            i_angle = np.pi - (i_angle + np.pi)
                        elif np.dot(aligned_unit_cell[i], principal_lattice_vector) > 0:
                            i_angle = -1 * i_angle
                    aligned_cell_angles = np.append(aligned_cell_angles, i_angle)  
                aligned_angle_list = list(aligned_cell_angles)
                aligned_cell_angles = set(aligned_cell_angles)
                cell_angle_combs = list(itertools.combinations(aligned_cell_angles, 2))
                comparison_translation_dirs = np.array([])
                for i in np.arange(0, len(cell_angle_combs)):
                    comparison_translation_dirs = np.append(combined_cell_angles, abs(sum(cell_angle_combs[i]) - (np.pi/2)))
                spanning_angle = min(comparison_translation_dirs)
                spanning_angle_index = list(comparison_translation_dirs).index(spanning_angle)
                constituent_angles = cell_angle_combs[spanning_angle_index]
                constituent_angle_indices = (aligned_angle_list.index(constituent_angles[0]), aligned_angle_list.index(constituent_angles[1]))
                choice = random.randint(0,1)
                secondary_lattice_angle = aligned_cell_angles[constituent_angle_indices[choice]]
                tertiary_lattice_angle = aligned_cell_angles[constituent_angle_indices[1 - choice]]
                secondary_lattice_vector = aligned_unit_cell[aligned_angle_list.index(secondary_lattice_angle)]
                tertiary_lattice_vector = aligned_unit_cell[aligned_angle_list.index(tertiary_lattice_angle)]
                aligned_spanning_set = np.array([principal_lattice_vector, secondary_lattice_vector, tertiary_lattice_vector])
                planar_spanning_set = []
                minimal_spanning_set = []
                for i in np.arange(0, len(aligned_spanning_set)):
                    planar_spanning_set = np.append(planar_spanning_set, aligned_spanning_set[i] @ coordinate_axis_reverse)
                for j in np.arange(0, len(aligned_spanning_set)):
                    minimal_spanning_set = np.append(minimal_spanning_set, planar_spanning_set[j] @ xy_plane_reverse)
                translational_symmetry_flags = np.zeros(len(minimal_spanning_set), len(nearest_neighbors))
                for i in np.arange(0, len(minimal_spanning_set)):
                    for j in np.arange(0, len(nearest_neighbors)):
                        adjusted_j_point = nearest_neighbors[j] + minimal_spanning_set[i]
                        for k in np.arange(0, len(bravais_lattice)):
                            if adjusted_j_point[0] == bravais_lattice[k][0] and adjusted_j_point[1] == bravais_lattice[k][1] and adjusted_j_point[0] == bravais_lattice[k][2]:
                                translational_symmetry_flags[i][j] == 1
                if 0 not in translational_symmetry_flags:
                    lattice_vectors = minimal_spanning_set
                    principal_lattice_vector = minimal_spanning_set[0]
            elif phase == 'gas':
                vacuum_between_cells = True
                if vacuum_between_cells == True:
                    vacuum_space = 10
                    atomic_pos = pd.read_csv(seed_structure)
                    xyz_pos = atomic_pos[['atom','x','y','z']]
                    structure_file = seed_structure_directory + "/" + molecular_formula + ".xyz"
                    atoms_in_cluster = []
                    corresponding_positions = []
                    with open(structure_file, 'w') as xyz_structure:
                        for index,row in xyz_pos.iterrows():
                            xyz_structure.write(f"{row['atom']}{row['x']} {row['y']} {row['z']}\n")
                            atoms_in_cluster.append(row['atom'])
                            corresponding_positions.append(list(np.array([row['x'], row['y'], row['z']])))
                    xyz_structure.close()
                    cluster = pmtg.core.Molecule(atoms_in_cluster, corresponding_positions)
                    cluster = cluster.get_centered_molecule()
                    structure_obj = pmtg.core.Structure(lattice = pmtg.core.Lattice.cubic(vacuum_space), species = cluster.species, coords = cluster.cart_coords, coords_are_cartesian = True)
                    calculation_directory = seed_structure_directory + "/relax"
                    incar_settings = {"EDIFF":1E-6, "EDIFFG":-0.02, "ISMEAR":0, "NSW": 200, "NPAR":8, "ISIF":2}
                    if input_format == "manual":
                        poscar = pmtg.io.vasp.Poscar(structure_obj)
                        symbols = atoms_in_cluster
                        #potcar = pmtg.io.vasp.Potcar([symbols], functional = xc_functional)
                        incar = pmtg.io.vasp.Incar(incar_settings)
                        reciporical_lattice_density = 100
                        kpoints = pmtg.io.vasp.Kpoints.automatic(reciporical_lattice_density)
                        poscar.write_file("POSCAR")
                        #potcar.write_file("POTCAR")
                        incar.write_file("INCAR")
                        kpoints.write_file("KPOINTS")
                        print('complete')
                        #archive_name = os.path.basename(calculation_directory)
                        #with tarfile.open(molecular_formula + "_inputs" + ".tar.gz", "w:gz") as tar_archive:
                            #tar_archive.add(calculation_directory, archive_name = archive_name)
                        #input_files = calculation_directory + "/" + molecular_formula + "_inputs" + ".tar.gz"
                    elif input_format == "input_set":
                        kpoints_settings = {"reciprocal_density":100}
                        custom_input = pmtg.io.vasp.sets.MPRelaxSet(structure_obj, user_incar_settings = incar_settings, user_kpoints_settings = kpoints_settings, user_potcar_functional = xc_functional)
                        custom_input.write_input(calculation_directory)
                        input_files = custom_input
        #return input_files
                

In [7]:
def find_atomic_symbol(element):
    characteristic_list = []
    blanks = np.array([])
    for i in np.arange(0, len(str(element))):
        characteristic_list.append(str(element)[i])
    for j in np.arange(0, len(characteristic_list)):
        if characteristic_list[j] == " ":
            blanks = np.append(blanks, j)
    atomic_symbol = str(element)[int(blanks[0]):int(blanks[1])]
    return atomic_symbol

atomic_numbers = np.array([])
element_set = ptable.get_all_elements()
for i in np.arange(0, len(element_set)):
    if find_atomic_symbol(element_set[i]) == find_atomic_symbol(element_set[56]):
        print('lanthanum')
        print(element_set[i].block)

lanthanum
d


In [10]:
#csv_filepath = "/users/haydenprescott/documents/test.csv"
#record_filepath = "/users/haydenprescott/test_data.tfrecord" 
#record_filename = "test_data.tfrecord"

#gaussian_input = g16_input_file(seed_structure_directory = "/users/haydenprescott/documents", molecular_formula = "La3O4", job_type = "ground_state_energy_calculation", input_directory = "/users/haydenprescott/documents", output_directory = "/users/haydenprescott/documents/gaussian_test_data", AO_basis = " ", polarization = None, job_title = "seed structure ground state", SCF_optimization = 'tight', csv_filepath = csv_filepath, record_filepath = record_filepath, record_filename = record_filename, element = None, n_CPU = 2, allowed_memory = "1000MB")
#print(gaussian_input.generate_input_file(molecular_formula = gaussian_input.molecular_formula, job_type = gaussian_input.job_type, input_directory = gaussian_input.input_directory, output_directory = gaussian_input.output_directory, seed_structure_directory = gaussian_input.seed_structure_directory, csv_filepath = gaussian_input.csv_filepath, record_filepath = gaussian_input.record_filepath, record_filename = gaussian_input.record_filename))
#vasp_input = VASP_input(working_directory = "/users/haydenprecott/documents/VASP_input_output", molecular_formula = "B7", job_type = "ground_state_energy_calculations", seed_structure_directory = "/users/haydenprescott/documents", csv_filepath = csv_filepath, record_filepath = record_filepath, record_filename = record_filename, pseudopotential = "", xc_functional = 'PBE64', cluster_type = 'periodic', principal_lattice_vector = np.array([]), accuracy_bound = 1e-3, max_iterations = 1000, complex_rotation_term = 0, rotation_angle = 0, n_atoms = 7, average_bond_length = 3.05, phase = 'gas')

#vasp_input.generate_cluster_object(seed_structure_directory = vasp_input.seed_structure_directory, molecular_formula = vasp_input.molecular_formula, cluster_type = vasp_input.cluster_type, csv_filepath = vasp_input.csv_filepath, record_filepath = vasp_input.record_filepath, record_filename = vasp_input.record_filename, accuracy_bound = vasp_input.accuracy_bound, max_iterations = vasp_input.max_iterations, n_atoms = vasp_input.n_atoms, average_bond_length = vasp_input.average_bond_length, phase = 'gas')
#xy_mapped_vector = vasp_input.find_mapping_to_coordinate_axis(principal_lattice_vector = np.array([-2, 3, 3]), accuracy_bound = vasp_input.accuracy_bound, max_iterations = vasp_input.max_iterations)
#print(vasp_input.generate_cluster_object(seed_structure_directory = vasp_input.seed_structure_directory, molecular_formula = vasp_input.molecular_formula, cluster_type = 'periodic', csv_filepath = vasp_input.csv_filepath, record_filepath = vasp_input.record_filepath, record_filename = vasp_input.record_filename, accuracy_bound = vasp_input.accuracy_bound, max_iterations = vasp_input.max_iterations, n_atoms = vasp_input.n_atoms))                             

In [72]:
csv_filepath = "/users/haydenprescott/documents/test.csv"
record_filepath = "/users/haydenprescott/test_data.tfrecord" 
record_filename = "test_data.tfrecord"
molecular_formula = "B7"
seed_structure_directory = "/users/haydenprescott/documents/VASP_input_output"

vasp_input = VASP_input(working_directory = "/users/haydenprecott/documents/VASP_input_output", molecular_formula = molecular_formula, job_type = "geometry_optimization", seed_structure_directory = seed_structure_directory, csv_filepath = csv_filepath, record_filepath = record_filepath, record_filename = record_filename, pseudopotential = "B_s", xc_functional = 'PBE_52', cluster_type = 'periodic', principal_lattice_vector = np.array([]), accuracy_bound = 1e-3, max_iterations = 1000, complex_rotation_term = 0, rotation_angle = 0, n_atoms = 7, average_bond_length = 3.05, phase = 'gas', input_format = 'manual', run_calculation = True)
print(vasp_input.generate_cluster_object(seed_structure_directory = vasp_input.seed_structure_directory, molecular_formula = vasp_input.molecular_formula, cluster_type = vasp_input.cluster_type, csv_filepath = vasp_input.csv_filepath, record_filepath = vasp_input.record_filepath, record_filename = vasp_input.record_filename, accuracy_bound = vasp_input.accuracy_bound, max_iterations = vasp_input.max_iterations, n_atoms = vasp_input.n_atoms, average_bond_length = vasp_input.average_bond_length, xc_functional = vasp_input.xc_functional, phase = vasp_input.phase, input_format = vasp_input.input_format, run_calculation = vasp_input.run_calculation))

#help(pmtg.io.vasp.sets.Poscar)
    

complete
None


/var/folders/3w/msxr0yxx71d4flsgsqvzh5c80000gn/T/ipykernel_50962/3637452643.py:334: DeprecationWarning: Please use INCAR KSPACING tag.
  kpoints = pmtg.io.vasp.Kpoints.automatic(reciporical_lattice_density)


In [50]:
    for j in np.arange(0, len(guess_cell)):
                if guess_cell[j] == representative_norms[unit_cell_size + j]:
                    cell_norms = np.append(cell_norms, guess_cell)
            cell_coordinates = bravais_lattice.transpose()[0:len(cell_norms) - 1]
            cell_coordinates = cell_coordinates.transpose()
            principal_cell_dimension = max(cell_norms)
            principal_lattice_vector = np.array([cell_coordinates[list(cell_norms).index(principal_cell_dimension)][0], cell_coordinates[list(cell_norms).index(principal_cell_dimension)][1], cell_coordinates[list(cell_norms).index(principal_cell_dimension)][2]])

 for j in range(0, len(aligned_unit_cell)):
                    if i != j:
                        i_norm = np.sqrt(aligned_unit_cell[i][0]**2 + aligned_unit_cell[i][1]**2 + aligned_unit_cell[i][2]**2)
                        j_norm = np.sqrt(aligned_unit_cell[j][0]**2 + aligned_unit_cell[j][1]**2 + aligned_unit_cell[j][2]**2)
                        i_j_angle = np.dot(aligned_unit_cell[i], aligned_unit_cell[j]) / (i_norm * j_norm)
                        if i_j_angle < 0:
                            if np.dot(aligned_unit_cell[i], aligned_unit_cell[j]) < 0:
                                i_j_angle = np.pi - (i_j_angle + np.pi)
                            elif np.dot(aligned_unit_cell[i], aligned_unit_cell[j]) > 0:
                                i_j_angle = -1 * i_j_angle
                        aligned_cell_angles = np.append(aligned_cell_angles, i_j_angle)
                    else:
                        continue

12.0


In [9]:

    
    
    
print(mapping_to_plane(principal_lattice_vector = np.array([-1,2,2]), accuracy_bound = 10e-5, max_iterations = 1000))

In [ ]:

def horizontal_mapping(prev_lattice_vector, max_iterations, accuracy_bound):
    principal_lattice_vector = np.array([prev_lattice_vector[0], prev_lattice_vector[1]])
    planar_shift = principal_lattice_vector[1]
    rotation_steps = []
    update_index = 0
    planar_transformations = 0
    while planar_transformations < max_iterations:
        planar_norm = np.sqrt(principal_lattice_vector[0]**2 + principal_lattice_vector[1]**2)
        unit_cell_axis = np.array([principal_lattice_vector[0], 0])
        cell_axis_norm = np.sqrt(unit_cell_axis[0]**2 + unit_cell_axis[1]**2)
        unit_cell_axis = np.array([unit_cell_axis[0] * (planar_norm / cell_axis_norm), unit_cell_axis[1] * (planar_norm / cell_axis_norm)])
        a = sympy.symbols('a', real = True)
        solution_space = sympy.solve(planar_norm**2 * sympy.cos(a) - np.dot(principal_lattice_vector, unit_cell_axis), a)
        if len(solution_space) > 0:
            xy_angular_shift = (2 * np.pi) - solution_space[0] 
            axis_of_rotation = np.array([0,0,1])
            imag_term = sympy.I
            rotation_number = sympy.cos(xy_angular_shift) + sympy.sin(xy_angular_shift) * imag_term
            rotation_matrix = convert_rotation_to_matrix(rotation_number, xy_angular_shift)
            checkpoint = principal_lattice_vector
            checkpoint_matrix = rotation_matrix
            principal_lattice_vector = principal_lattice_vector @ rotation_matrix
            new_shift = principal_lattice_vector[1]
            if abs(new_shift) < abs(planar_shift):
                rotation_steps.append(rotation_matrix)
                planar_transformations += 1
                planar_shift = new_shift
            else:
                xy_angular_shift = -1 * ((2 * np.pi) - solution_space[0])
                axis_of_rotation = np.array([0,0,1])
                rotation_number = sympy.cos(xy_angular_shift) + sympy.sin(xy_angular_shift) * imag_term
                rotation_matrix = convert_rotation_to_matrix(rotation_number, xy_angular_shift)
                principal_lattice_vector = checkpoint @ rotation_matrix
                new_shift = principal_lattice_vector[1]
                n_transformations += 1
                if abs(new_shift) < abs(planar_shift):
                    rotation_steps.append(rotation_matrix)
                    planar_transformations += 1
                    planar_shift = new_shift
                else:
                    if planar_shift > 0:
                        variation_angle = -1 * (np.pi / 180)
                    else:
                        variation_angle = np.pi / 180
                    axis_of_rotation = np.array([0,0,1])
                    checkpoint_norm = sympy.sqrt(checkpoint[0]**2 + checkpoint[1]**2)
                    variation_number = np.cos(variation_angle) + np.sin(variation_angle) * imag_term
                    variation_matrix = convert_rotation_to_matrix(variation_number, variation_angle)
                    varied_components = checkpoint @ variation_matrix
                    rotation_steps.append(variation_matrix)
                    principal_lattice_vector = varied_components
                    planar_shift = principal_lattice_vector[1]
                    planar_transformations += 1
        if abs(planar_shift) >= abs(accuracy_bound):
            continue
        else:
            break  

    axis_alignment = sympy.eye(len(rotation_steps[0]))
    for i in np.arange(0, len(rotation_steps)):
        axis_alignment = axis_alignment @ sympy.Matrix(rotation_steps[i])
    u, s, v = axis_alignment.singular_value_decomposition()
    s = np.diag(np.array(s))
    reciporical_singular_vals = np.eye(len(s))
    for i in np.arange(0, len(s)):
        for j in np.arange(0, len(s)):
           if reciporical_singular_vals[i][j] == 1:
                reciporical_singular_vals[i][j] = 1 / s[j]
    reciporical_singular_vals = sympy.Matrix(reciporical_singular_vals)
    u_inv = u.inv()
    v_inv_t = u.inv().transpose()
    reverse_axis_alignment = u_inv @ reciporical_singular_vals @ v_inv_t
    return axis_alignment, reverse_axis_alignment

print(horizontal_mapping(prev_lattice_vector = xy_mapped_vector, max_iterations = 10e3, accuracy_bound = 10e05)[0], '\n', horizontal_mapping(prev_lattice_vector = xy_mapped_vector, max_iterations = 10e3, accuracy_bound = 10e05)[1])
      

In [11]:
vec = np.array([1,3,5])
b = sympy.symbols('b', real = True)
vecmag = sympy.sqrt(vec[0]**2 + vec[1]**2 + vec[2]**2)
varied_vec = np.array([b * vec[0], b * vec[1], vec[2] * 0.9])
varied_mag = sympy.sqrt(varied_vec[0]**2 + varied_vec[1]**2 + varied_vec[2]**2)
b_val = sympy.solve(varied_mag - vecmag, b)
print(b_val)

[-1.21449578014911, 1.21449578014911]


In [63]:
ar = np.array([1,2,3])
ar = set(ar)
combs = list(itertools.combinations(ar, 2))
print(sum(combs[1]))

4


In [60]:
help(pmtg.io.vasp.Potcar)

Help on class Potcar in module pymatgen.io.vasp.inputs:

class Potcar(builtins.list, monty.json.MSONable)
 |  Potcar(symbols: 'Sequence[str] | None' = None, functional: 'str | None' = None, sym_potcar_map: 'dict[str, str] | None' = None) -> 'None'
 |  
 |  Read and write POTCAR files for calculations. Consists of a list of PotcarSingle.
 |  
 |  Method resolution order:
 |      Potcar
 |      builtins.list
 |      monty.json.MSONable
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __init__(self, symbols: 'Sequence[str] | None' = None, functional: 'str | None' = None, sym_potcar_map: 'dict[str, str] | None' = None) -> 'None'
 |      Args:
 |          symbols (list[str]): Element symbols for POTCAR. This should correspond
 |              to the symbols used by VASP. e.g. "Mg", "Fe_pv", etc.
 |          functional (str): Functional used. To know what functional options
 |              there are, use Potcar.FUNCTIONAL_CHOICES. Note that VASP has
 |              different ve